In [1]:
import pandas as pd
import numpy as np


In [2]:
# Cargar las bases de datos
path_maestra = r"C:\Users\Rodrigo\Desktop\Roxana Files\base_maestra\base_maestra.xlsx"
path_anid = r"C:\Users\Rodrigo\Desktop\Roxana Files\Add databases\Github ANID\BDH_HISTORICA.xlsx"

# Leer los datos
maestra = pd.read_excel(path_maestra)
anid = pd.read_excel(path_anid)

print(f"Base maestra cargada con {maestra.shape[0]} filas y {maestra.shape[1]} columnas.")
print(f"Base ANID cargada con {anid.shape[0]} filas y {anid.shape[1]} columnas.")


Base maestra cargada con 3166 filas y 23 columnas.
Base ANID cargada con 44143 filas y 23 columnas.


In [3]:
# Filtrar la base ANID para los proyectos relevantes
anid_filtrada = anid[
    (anid['PROGRAMA'] == 'FONDECYT') &
    (anid['INSTRUMENTO'].isin(['REGULAR', 'POSTDOCTORADO', 'INICIACION'])) &
    (anid['AGNO_FALLO'].between(2014, 2024))
]

print(f"Base ANID filtrada con {anid_filtrada.shape[0]} filas.")

Base ANID filtrada con 13355 filas.


In [4]:
# Rellenar valores nulos en nombres
columnas_nombres = ['Primer_Nombre', 'Segundo_Nombre', 'Apellido_Paterno', 'Apellido_Materno']
maestra[columnas_nombres] = maestra[columnas_nombres].fillna('')

# Crear combinaciones de nombres
maestra = maestra.assign(
    comb_1=maestra['Primer_Nombre'] + " " + maestra['Segundo_Nombre'] + " " + maestra['Apellido_Paterno'] + " " + maestra['Apellido_Materno'],
    comb_2=maestra['Primer_Nombre'] + " " + maestra['Segundo_Nombre'] + " " + maestra['Apellido_Materno'] + " " + maestra['Apellido_Paterno'],
    comb_3=maestra['Primer_Nombre'] + " " + maestra['Apellido_Paterno'] + " " + maestra['Apellido_Materno'],
    comb_4=maestra['Primer_Nombre'] + " " + maestra['Apellido_Materno'] + " " + maestra['Apellido_Paterno'],
    comb_5=maestra['Segundo_Nombre'] + " " + maestra['Primer_Nombre'] + " " + maestra['Apellido_Paterno'] + " " + maestra['Apellido_Materno'],
    comb_7=maestra['Segundo_Nombre'] + " " + maestra['Apellido_Paterno'] + " " + maestra['Apellido_Materno'],
    comb_8=maestra['Segundo_Nombre'] + " " + maestra['Apellido_Materno'] + " " + maestra['Apellido_Paterno'],
    comb_11=maestra['Primer_Nombre'] + " " + maestra['Segundo_Nombre'] + " " + maestra['Apellido_Paterno']
)

# Crear columna identificadora para vincular registros
maestra['nombre_original_maestra'] = maestra['Primer_Nombre'] + " " + maestra['Apellido_Paterno']

print(f"Combinaciones generadas: {', '.join([col for col in maestra.columns if 'comb_' in col])}")


Combinaciones generadas: comb_1, comb_2, comb_3, comb_4, comb_5, comb_7, comb_8, comb_11


In [5]:
# Inicializar la base combinada
base_combinada = pd.DataFrame()

# Iterar sobre las combinaciones para realizar los merges
for comb_col in [col for col in maestra.columns if 'comb_' in col]:
    print(f"Realizando merge con combinación: {comb_col}")
    resultado = pd.merge(
        maestra,
        anid_filtrada,
        how='left',
        left_on=comb_col,
        right_on='NOMBRE_RESPONSABLE',
        suffixes=('', f'_{comb_col}')
    )
    base_combinada = pd.concat([base_combinada, resultado], ignore_index=True)

print(f"Base combinada creada con {base_combinada.shape[0]} filas.")


Realizando merge con combinación: comb_1
Realizando merge con combinación: comb_2
Realizando merge con combinación: comb_3
Realizando merge con combinación: comb_4
Realizando merge con combinación: comb_5
Realizando merge con combinación: comb_7
Realizando merge con combinación: comb_8
Realizando merge con combinación: comb_11
Base combinada creada con 25348 filas.


In [6]:
# Mantener filas únicas basadas en 'nombre_original_maestra' y 'NOMBRE_RESPONSABLE'
base_distinct = base_combinada.drop_duplicates(subset=['nombre_original_maestra', 'NOMBRE_RESPONSABLE'])
print(f"Base distinta creada con {base_distinct.shape[0]} filas únicas.")


Base distinta creada con 3828 filas únicas.


In [7]:
# Identificar becarios con proyectos (1 o más)
base_con_proyectos = base_distinct[base_distinct['NOMBRE_RESPONSABLE'].notnull()].copy()
base_con_proyectos['Tiene_fondecyt'] = 1

# Identificar becarios sin proyectos
becarios_sin_proyectos = maestra[~maestra['nombre_original_maestra'].isin(base_con_proyectos['nombre_original_maestra'])].copy()
becarios_sin_proyectos = becarios_sin_proyectos.assign(
    Tiene_fondecyt=0,
    cuantos_proyectos=0
)

# Unir ambas bases
base_final = pd.concat([base_con_proyectos, becarios_sin_proyectos], ignore_index=True)

# Validar integridad
assert base_final.shape[0] == maestra.shape[0], \
    f"Error: La base final no coincide con la base maestra. {base_final.shape[0]} != {maestra.shape[0]}"

print(f"Base final creada con {base_final.shape[0]} filas.")


AssertionError: Error: La base final no coincide con la base maestra. 3165 != 3166

In [ ]:
print(f"Total de nombres únicos en maestra: {maestra['nombre_original_maestra'].nunique()}")
print(f"Total de nombres únicos en base_final: {base_final['nombre_original_maestra'].nunique()}")


Total de nombres únicos en maestra: 3061
Total de nombres únicos en base_final: 3061


In [9]:
# Identificar nombres presentes en maestra pero no en base_final
nombres_faltantes = maestra.loc[
    ~maestra['nombre_original_maestra'].isin(base_final['nombre_original_maestra']),
    'nombre_original_maestra'
]
print(f"Nombres faltantes:\n{nombres_faltantes}")


Nombres faltantes:
Series([], Name: nombre_original_maestra, dtype: object)


In [10]:
registros_faltantes = maestra.loc[
    maestra['nombre_original_maestra'].isin(nombres_faltantes)
].copy()

# Asegurarnos de asignar valores adecuados
registros_faltantes = registros_faltantes.assign(
    Tiene_fondecyt=0,
    cuantos_proyectos=0
)

# Unir los registros faltantes a la base final
base_final = pd.concat([base_final, registros_faltantes], ignore_index=True)


In [11]:
assert base_final.shape[0] == maestra.shape[0], \
    f"Error: La base final aún no coincide con la base maestra. {base_final.shape[0]} != {maestra.shape[0]}"
print(f"Base final corregida con {base_final.shape[0]} filas.")


AssertionError: Error: La base final aún no coincide con la base maestra. 3165 != 3166

In [12]:
duplicados = base_final[base_final.duplicated(subset='nombre_original_maestra', keep=False)]
print(f"Total de duplicados en base_final: {duplicados.shape[0]}")
if not duplicados.empty:
    print("Duplicados detectados:")
    print(duplicados)


Total de duplicados en base_final: 205
Duplicados detectados:
         FOLIO                              NOMBRE Primer_Nombre  \
4     21140409             ALEJANDRO MIRANDA CERPA     ALEJANDRO   
20    21150288        BRIAN ALEXANDER EFFER ROLDAN         BRIAN   
24    72150023       CAMILA PATRICIA MOYANO DAVILA        CAMILA   
27    21140424       CARLOS ALEJANDRO CRUZ HERRERA        CARLOS   
42    21161211   CHRISTIAN JAVIER SANTANDER CASTRO     CHRISTIAN   
...        ...                                 ...           ...   
2944  21140490  SEBASTIAN ANDRES SALGADO REBOLLEDO     SEBASTIAN   
2971  72150510       SERGIO ANDRES CONTRERAS PINTO        SERGIO   
2975  21140057     SERGIO ANTONIO CONTRERAS HANTKE        SERGIO   
3130  72170181         XIMENA CAROLINA CANELO PINO        XIMENA   
3131  72170181         XIMENA CAROLINA CANELO PINO        XIMENA   

     Segundo_Nombre Apellido_Paterno Apellido_Materno SEXO_COMPLETO  \
4                            MIRANDA            CE

In [13]:
print(f"Total de filas en la maestra: {maestra.shape[0]}")
print(f"Total de filas en la base final: {base_final.shape[0]}")


Total de filas en la maestra: 3166
Total de filas en la base final: 3165


In [14]:
faltantes = maestra[~maestra['nombre_original_maestra'].isin(base_final['nombre_original_maestra'])]
print(f"Total de nombres faltantes: {faltantes.shape[0]}")
if not faltantes.empty:
    print("Nombres faltantes en base_final:")
    print(faltantes)

Total de nombres faltantes: 0


In [15]:
# Resolver duplicados en la base final
print("Resolviendo duplicados...")

# Ordenar por `AGNO_FALLO` descendente para priorizar el proyecto más reciente
base_final = base_final.sort_values(by=['nombre_original_maestra', 'AGNO_FALLO'], ascending=[True, False])

# Eliminar duplicados manteniendo solo uno por nombre
base_final = base_final.drop_duplicates(subset='nombre_original_maestra', keep='first')

# Volver a calcular la columna `cuantos_proyectos` para reflejar la cantidad total de proyectos por becario
base_final['cuantos_proyectos'] = base_final.groupby('nombre_original_maestra')['NOMBRE_RESPONSABLE'].transform('count')

# Validar que ahora el tamaño coincide con la base maestra
assert base_final.shape[0] == maestra.shape[0], \
    f"Error: La base final aún no coincide con la base maestra. {base_final.shape[0]} != {maestra.shape[0]}"

print(f"Base final corregida con {base_final.shape[0]} filas.")
print(f"Distribución actualizada de 'Tiene_fondecyt':")
print(base_final['Tiene_fondecyt'].value_counts())


Resolviendo duplicados...


AssertionError: Error: La base final aún no coincide con la base maestra. 3061 != 3166

In [16]:
# Identificar los nombres únicos en la maestra y en la base final
nombres_maestra = set(maestra['nombre_original_maestra'])
nombres_final = set(base_final['nombre_original_maestra'])

# Nombres que están en la maestra pero no en la base final
nombres_faltantes = nombres_maestra - nombres_final

# Mostrar resultados
print(f"Total de nombres únicos en la maestra: {len(nombres_maestra)}")
print(f"Total de nombres únicos en la base final: {len(nombres_final)}")
print(f"Total de nombres faltantes: {len(nombres_faltantes)}")

# Mostrar los nombres faltantes
print("Nombres faltantes:")
print(nombres_faltantes)


Total de nombres únicos en la maestra: 3061
Total de nombres únicos en la base final: 3061
Total de nombres faltantes: 0
Nombres faltantes:
set()


In [18]:
# Identificar proyectos adicionales (becarios con más de un proyecto en ANID)
proyectos_adicionales = base_distinct[base_distinct.duplicated(subset='nombre_original_maestra', keep=False)].copy()

# Verificar cuántos proyectos adicionales hay
print(f"Total de proyectos adicionales: {proyectos_adicionales.shape[0]}")


Total de proyectos adicionales: 1479


In [19]:
# Ruta para exportar el archivo
output_path = "base_maestra_con_proyectos.xlsx"

# Exportar base final y proyectos adicionales a diferentes hojas
with pd.ExcelWriter(output_path) as writer:
    base_final.to_excel(writer, sheet_name="Base Final", index=False)
    proyectos_adicionales.to_excel(writer, sheet_name="Proyectos Adicionales", index=False)

print(f"Base de datos exportada a {output_path}")


Base de datos exportada a base_maestra_con_proyectos.xlsx
